In [1]:
import pandas as pd
import os
import json
from collections import defaultdict
os.getcwd()

'D:\\pgkb_graph'

In [2]:
diplotype_folder = "diplotype"

mentioned_genes = ["CYP2B6", "CYP2C9", "CYP2C19", "CYP2D6", "CYP3A5",
                   "DPYD", "NUDT15", "SLCO1B1", "TPMT", "UGT1A1"]

diplotype_path_dict = {g: "" for g in mentioned_genes}
for path in os.listdir(diplotype_folder):
    for gene in mentioned_genes:
        if "{}_Diplotype".format(gene) in path:
            diplotype_path_dict[gene] = os.path.join(diplotype_folder, path)
for key,value in sorted(diplotype_path_dict.items(), key=lambda x: x[0]):
    print(key, value)

CYP2B6 diplotype\CYP2B6_Diplotype_Phenotype_Table.xlsx
CYP2C19 diplotype\CYP2C19_Diplotype_Phenotype_Table.xlsx
CYP2C9 diplotype\CYP2C9_Diplotype_Phenotype_Table.xlsx
CYP2D6 diplotype\CYP2D6_Diplotype_Phenotype_Table.xlsx
CYP3A5 diplotype\CYP3A5_Diplotype_Phenotype_Table.xlsx
DPYD diplotype\DPYD_Diplotype_Phenotype_Table.xlsx
NUDT15 diplotype\NUDT15_Diplotype_Phenotype_Table.xlsx
SLCO1B1 diplotype\SLCO1B1_Diplotype_Phenotype_Table.xlsx
TPMT diplotype\TPMT_Diplotype_Phenotype_Table.xlsx
UGT1A1 diplotype\UGT1A1_Diplotype_Phenotype_Table.xlsx


In [27]:
def parse_diplotype(path):
    xl = pd.ExcelFile(path)
    
    diplotype_dict = {}
    phenotype_dict = {}
    for sheet in xl.sheet_names:
        if "diplotype" in sheet.lower():
            ### read and handle diplotype
            df_diplotype = pd.read_excel(xl, sheet_name=sheet).fillna("")
            diplotype_column = ""
            phenotype_column = ""
            ehr_notation_column = ""
            score_column = ""
            variant_1_column = ""
            variant_2_column = ""

            for column in df_diplotype.columns:
                if "diplotype" in column.lower() and diplotype_column == "":
                    diplotype_column = column
                elif "phenotype" in column.lower() or "metabolizer" in column.lower() and phenotype_column == "":
                    phenotype_column = column
                elif "notation" in column.lower():
                    ehr_notation_column = column
                elif "score" in column.lower():
                    score_column = column
                elif "variant" in column.lower() and "1" in column:
                    variant_1_column = column
                elif "variant" in column.lower() and "2" in column:
                    variant_2_column = column

            for index, row in df_diplotype.iterrows():
                if diplotype_column != "":
                    diplotype = row[diplotype_column]
                elif variant_1_column != "" and variant_2_column != "":
                    diplotype = "{}/{}".format(row[variant_1_column], row[variant_2_column])
                else:
                    continue

                phenotype = row[phenotype_column] if phenotype_column != "" else ""
                ehr_notation = row[ehr_notation_column] if ehr_notation_column != "" else ""
                score = row[score_column] if score_column != "" else ""
                
                if "{}{}{}".format(phenotype, ehr_notation, score) != "":
                    diplotype_dict[diplotype] = {
                        "phenotype": phenotype,
                        "ehr_notation": ehr_notation,
                        "score": score
                    }

        elif "consult note" in sheet.lower():
            ### read and handle consult note
            df_consult = pd.read_excel(xl, sheet_name=sheet).fillna("")
            phenotype_column = ""
            ehr_notation_column = ""
            consultation_column = ""
            score_column = ""
            for column in df_consult.columns:
                if "phenotype" in column.lower() or "metabolizer" in column.lower():
                    phenotype_column = column
                elif "notation" in column.lower():
                    ehr_notation_column = column
                elif "consultation" in column.lower():
                    consultation_column = column
                elif "score" in column.lower():
                    score_column = column

            for index, row in df_consult.iterrows():
                phenotype = row[phenotype_column]
                
                consultation = row[consultation_column] if consultation_column != "" else ""
                ehr_notation = row[ehr_notation_column] if ehr_notation_column != "" else ""
                score = row[score_column] if score_column != "" else ""
                
                if "{}{}{}".format(consultation, ehr_notation, score) != "":
                    phenotype_dict[phenotype] = {
                        "ehr_notation": ehr_notation,
                        "consultation": consultation,
                        "score": score
                    }
    return diplotype_dict, phenotype_dict

for key, value in sorted(diplotype_path_dict.items(), key=lambda x: x[0]):
    diplotype_dict, phenotype_dict = parse_diplotype(value)
    print(key)
    print(diplotype_dict)
    print(key)
    print(phenotype_dict)

CYP2B6
{'*1/*1': {'phenotype': 'CYP2B6 Normal Metabolizer ', 'ehr_notation': 'Normal/Routine/Low Risk', 'score': ''}, '*1/*2': {'phenotype': 'CYP2B6 Normal Metabolizer ', 'ehr_notation': 'Normal/Routine/Low Risk', 'score': ''}, '*1/*3': {'phenotype': 'Indeterminate', 'ehr_notation': 'None', 'score': ''}, '*1/*4': {'phenotype': 'CYP2B6 Rapid Metabolizer', 'ehr_notation': 'Normal/Routine/Low Riskc', 'score': ''}, '*1/*5': {'phenotype': 'CYP2B6 Normal Metabolizer ', 'ehr_notation': 'Normal/Routine/Low Risk', 'score': ''}, '*1/*6': {'phenotype': 'CYP2B6 Intermediate Metabolizer', 'ehr_notation': 'Abnormal/Priority/High Risk', 'score': ''}, '*1/*7': {'phenotype': 'CYP2B6 Intermediate Metabolizer', 'ehr_notation': 'Abnormal/Priority/High Risk', 'score': ''}, '*1/*8': {'phenotype': 'CYP2B6 Intermediate Metabolizer', 'ehr_notation': 'Abnormal/Priority/High Risk', 'score': ''}, '*1/*9': {'phenotype': 'CYP2B6 Intermediate Metabolizer', 'ehr_notation': 'Abnormal/Priority/High Risk', 'score': ''},

CYP2C9
{'*3/*3': {'phenotype': 'CYP2C9 Poor Metabolizer ', 'ehr_notation': 'Abnormal/Priority/High Risk', 'score': 0.0}, '*3/*6': {'phenotype': 'CYP2C9 Poor Metabolizer ', 'ehr_notation': 'Abnormal/Priority/High Risk', 'score': 0.0}, '*3/*13': {'phenotype': 'CYP2C9 Poor Metabolizer ', 'ehr_notation': 'Abnormal/Priority/High Risk', 'score': 0.0}, '*3/*15': {'phenotype': 'CYP2C9 Poor Metabolizer ', 'ehr_notation': 'Abnormal/Priority/High Risk', 'score': 0.0}, '*3/*24': {'phenotype': 'CYP2C9 Poor Metabolizer ', 'ehr_notation': 'Abnormal/Priority/High Risk', 'score': 0.0}, '*3/*25': {'phenotype': 'CYP2C9 Poor Metabolizer ', 'ehr_notation': 'Abnormal/Priority/High Risk', 'score': 0.0}, '*3/*33': {'phenotype': 'CYP2C9 Poor Metabolizer ', 'ehr_notation': 'Abnormal/Priority/High Risk', 'score': 0.0}, '*3/*35': {'phenotype': 'CYP2C9 Poor Metabolizer ', 'ehr_notation': 'Abnormal/Priority/High Risk', 'score': 0.0}, '*3/*39': {'phenotype': 'CYP2C9 Poor Metabolizer ', 'ehr_notation': 'Abnormal/Prio

CYP2D6
{'*1/*1': {'phenotype': 'Normal Metabolizer', 'ehr_notation': 'Normal/Routine/ Low Risk', 'score': '2'}, '*1/*1x2': {'phenotype': 'Ultrarapid Metabolizer', 'ehr_notation': 'Abnormal/Priority/High Risk', 'score': '3'}, '*1/*1≥3': {'phenotype': 'Ultrarapid Metabolizer', 'ehr_notation': 'Abnormal/Priority/High Risk', 'score': '≥4'}, '*1/*2': {'phenotype': 'Normal Metabolizer', 'ehr_notation': 'Normal/Routine/ Low Risk', 'score': '2'}, '*1/*2x2': {'phenotype': 'Ultrarapid Metabolizer', 'ehr_notation': 'Abnormal/Priority/High Risk', 'score': '3'}, '*1/*2≥3': {'phenotype': 'Ultrarapid Metabolizer', 'ehr_notation': 'Abnormal/Priority/High Risk', 'score': '≥4'}, '*1/*3': {'phenotype': 'Intermediate Metabolizer', 'ehr_notation': 'Abnormal/Priority/High Risk', 'score': '1'}, '*1/*3x2': {'phenotype': 'Intermediate Metabolizer', 'ehr_notation': 'Abnormal/Priority/High Risk', 'score': '1'}, '*1/*4': {'phenotype': 'Intermediate Metabolizer', 'ehr_notation': 'Abnormal/Priority/High Risk', 'sco

TPMT
{'*1/*2': {'phenotype': 'TPMT Intermediate Metabolizer', 'ehr_notation': 'Abnormal/Priority/High Risk', 'score': ''}, '*1/*3A': {'phenotype': 'TPMT Intermediate Metabolizer', 'ehr_notation': 'Abnormal/Priority/High Risk', 'score': ''}, '*1/*3B': {'phenotype': 'TPMT Intermediate Metabolizer', 'ehr_notation': 'Abnormal/Priority/High Risk', 'score': ''}, '*1/*3C': {'phenotype': 'TPMT Intermediate Metabolizer', 'ehr_notation': 'Abnormal/Priority/High Risk', 'score': ''}, '*1/*4': {'phenotype': 'TPMT Intermediate Metabolizer', 'ehr_notation': 'Abnormal/Priority/High Risk', 'score': ''}, '*1/*11': {'phenotype': 'TPMT Intermediate Metabolizer', 'ehr_notation': 'Abnormal/Priority/High Risk', 'score': ''}, '*1/*14': {'phenotype': 'TPMT Intermediate Metabolizer', 'ehr_notation': 'Abnormal/Priority/High Risk', 'score': ''}, '*1/*15': {'phenotype': 'TPMT Intermediate Metabolizer', 'ehr_notation': 'Abnormal/Priority/High Risk', 'score': ''}, '*1/*23': {'phenotype': 'TPMT Intermediate Metaboliz